In [57]:
import pandas as pd
import pickle
import numpy as np
from stellargraph import StellarGraph
import os
import networkx as nx
import numpy as np
import pandas as pd

from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph

In [63]:
os.chdir('/home/anoopkumar/')
all_sub_adj_mat = []
all_sub_label = []

with open ('all_subject_adj_mat.pkl','rb') as f1, open('all_subject_label.pkl','rb') as f2:
    all_sub_adj_mat = pickle.load(f1)
    all_sub_label = pickle.load(f2)

In [66]:
newall = []
for sub in all_sub_adj_mat:
    newsub = []
    for row in sub:
        row = np.append(row,1)
        newsub.append(row)
    newsub = np.vstack([newsub,[1 for i in range(91)]])
    newall.append(newsub)



In [67]:
print(len(all_sub_label))
print(len(all_sub_adj_mat))
print(len(all_sub_adj_mat[0]))
print(len(all_sub_adj_mat[0][0]))
print(len(newall))
print(len(newall[0]))
print(len(newall[0][0]))

160
160
90
90
160
91
91


In [69]:
all_sg = [[] for i in range(160)]

In [71]:
for count, sub in enumerate(newall):
    np.fill_diagonal(sub, np.nan)
    df = pd.DataFrame(sub)
    df = df.stack().reset_index()
    pairs = list(zip(df['level_0'], df['level_1']))
    for i in pairs:
            a, b = i
            if (b,a) not in all_sg[count]:
                all_sg[count].append(i)

In [72]:
all_sg_lr = [[] for _ in range(160)]

for i,x in enumerate(all_sg):
    all_sg_lr[i] = [[],[]]
    for pair in x:
        a, b = pair
        all_sg_lr[i][0].append(a)
        all_sg_lr[i][1].append(b)

In [89]:
graphs = []

for one_sglr in all_sg_lr:
    dic = {"source": one_sglr[0],"target":one_sglr[1]}
    one = pd.DataFrame(dic)
    graphs.append(StellarGraph(edges = one))

In [96]:
print(graphs[0].info()) 
G = graphs

StellarGraph: Undirected multigraph
 Nodes: 91, Edges: 4095

 Node types:
  default: [91]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [4095]
        Weights: all 1 (default)
        Features: none


In [233]:
allwalk = []
for sub in G:
    rw = BiasedRandomWalk(sub)
    walks = rw.run(
    nodes=list(sub.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
    )
    allwalk.append(walks)
# print("Number of random walks: {}".format(len(walks)))


In [234]:
print(len(allwalk))

160


In [188]:
modelvecs = []
from gensim.models import Word2Vec

for walks in allwalk:
    str_walks = [[str(n) for n in walk] for walk in walks]
    model = Word2Vec(str_walks, size=80, window=5, min_count=0, sg=1, workers=2, iter=1)
    modelvecs.append(model)

In [189]:
graphembeds = []

for vecs in modelvecs:
    node_embeds = (vecs.wv.vectors)
    print(len(node_embeds))
    break

91


In [190]:
supernode = []
avgnode = []

for vecs in modelvecs:
    node_embeds = (vecs.wv.vectors)
    supernode.append(node_embeds[-1])
    avgnode.append(node_embeds.mean(axis = 0)) 

In [215]:
print(len(supernode),len(avgnode))
graph_labels = []
rem_ad = []
for c,i in enumerate(all_sub_label):
    if i == 'CN':
        graph_labels.append(1)
    elif i == 'MCI' or i == 'EMCI' or i == 'LMCI':
        graph_labels.append(-1)
    else:
        rem_ad.append(c)

exp1 = []
for i in range(160):
    if i not in rem_ad:
        exp1.append(supernode[i])
        
print(len(exp1), len (graph_labels))

160 160
129 129


In [ ]:
############# supernode check #############################################

In [216]:
data = exp1
label = graph_labels
print(len(data),len(label))

129 129


In [217]:
pd.DataFrame(label).value_counts().to_frame()

,0
0,
-1,89
1,40


In [218]:
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
clf1 = SVC(gamma='auto')
# clf2 =make_pipeline(StandardScaler(), MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(16, 4), random_state=1))
clf3 = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0,max_depth=5, random_state=0)

X = np.array(data)
y = np.array(label)


In [220]:
cv_results = cross_validate(clf1, X, y, cv=40, scoring = ['accuracy','roc_auc','balanced_accuracy'])
# cv_results2 = cross_validate(clf2, X, y, cv=40)
cv_results3 = cross_validate(clf3, X, y, cv=40,scoring = ['accuracy','roc_auc','balanced_accuracy'])

In [232]:
print(np.mean(cv_results['test_roc_auc']))
# print((cv_results['test_score']))
print(np.mean(cv_results3['test_roc_auc']))
print(np.mean(cv_results3['test_balanced_accuracy']))


0.45
0.4916666666666667
0.4875
